In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
trip2021 = pd.read_csv(r'.\APC_Data\All_Routes_Trip_Summary_Summer2021.csv')

C:\Users\brine\miniconda3\envs\musa-550-fall-2021\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
list(trip2021.columns)

['Unnamed: 0',
 'Route',
 'Direction',
 'Pattern',
 'Trip',
 'Trip Code',
 'Trip Type',
 'Stop Code',
 'Stop Seq.',
 'Avg. Ons',
 'Avg. Offs',
 'Load',
 'Dwell Time',
 'Sch. Start Time',
 'Sch. End Time',
 'Act. Start Time',
 'Act. End Time',
 'Time Dev',
 'Sch. Run Time',
 'Act. Run Time',
 'Run Dev']

In [4]:
trip2021 = trip2021.drop(columns=["Unnamed: 0"])

In [5]:
trip2021.head()

,Route,Direction,Pattern,Trip,Trip Code,Trip Type,Stop Code,Stop Seq.,Avg. Ons,Avg. Offs,Load,Dwell Time,Sch. Start Time,Sch. End Time,Act. Start Time,Act. End Time,Time Dev,Sch. Run Time,Act. Run Time,Run Dev
0,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,4st,(001) - Roosevelt Blvd & Broad St - FS,9.696970,0.000000,9.696970,00:01:27,05:13:00,05:13:00,05:13:26,05:09:31,00:00:26,00:01:26,00:02:10,00:00:44
1,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,22406st,(002) - Roosevelt Blvd & 9th St,2.303030,0.000000,12.000000,00:00:14,05:14:26,05:14:26,05:15:10,05:14:25,00:00:44,00:00:52,00:01:03,00:00:11
2,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,23308st,(003) - Roosevelt Blvd & Wyoming Av,0.000000,0.000000,12.000000,00:00:00,05:15:18,05:15:18,05:15:55,05:15:38,00:00:37,00:00:21,00:00:25,00:00:04
3,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,5st,(004) - Roosevelt Blvd & 5th St,2.515152,0.090909,14.424242,00:00:13,05:15:39,05:15:39,05:16:39,05:15:55,00:01:00,00:01:15,00:01:46,00:00:31
4,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,22407st,(005) - Roosevelt Blvd & Mascher St,0.818182,0.000000,15.242424,00:00:07,05:16:54,05:16:54,05:18:07,05:17:41,00:01:13,00:00:18,00:00:19,00:00:01


In [24]:
codes=trip2021["Trip Code"].unique()

In [25]:
for i in codes:
    print(i)

8548782
8548781
8548780
8548779
8548777
8548778
8548776
8548775
8548771
8548774
8548797
8548796
8548795
8548792
8548791
8548793
8548773
8548794
8548772
8548790
8548789
8548788
8548787
8548783
8548786
8548785
8548784
8548813
8548812
8548811
8548810
8548809
8548821
8548808
8548807
8548806
8548805
8548804
8548803
8548802
8548822
8548819
8548801
8548820
8548798
8548823
8548800
8548814
8548799
8548818
8548817
8548816
8548815
8548945
8548946
8548947
8548948
8548949
8548950
8548951
8548952
8548953
8548954
8548955
8548956
8548958
8548957
8548935
8548932
8548933
8548960
8548962
8548963
8548964
8548904
8548914
8548905
8548906
8548907
8548908
8548909
8548910
8548911
8548912
8548913
8548959
8548961
8548915
8548916
8548917
8548918
8548919
8548922
8548920
8548924
8548923
8548927
8548925
8548929
8548928
8548926
8548930
8548921
8548931
8548936
8548937
8548934
8548938
8548939
8548940
8548941
8548942
8548943
8548944
8549015
8548980
8548977
8548978
8548979
8548981
8548982
8548983
8548984
8548985
8549018


8561850
8561851
8561852
8561853
8561854
8561855
8561848
8561856
8561810
8561857
8561858
8561859
8561860
8561861
8561863
8561862
8561812
8561811
8561813
8561814
8561815
8561816
8561817
8561818
8561819
8561820
8561821
8561822
8561823
8561824
8561825
8561877
8561879
8561870
8561872
8561873
8561875
8561874
8561883
8561876
8561884
8561885
8561886
8561887
8561889
8561890
8561891
8561892
8561882
8561893
8561894
8561895
8561896
8561898
8561899
8561900
8561897
8561880
8561868
8561878
8561901
8561903
8561902
8561905
8561904
8561906
8561907
8561908
8561881
8561865
8561866
8561867
8561888
8561909
8561910
8561911
8561912
8561913
8561914
8561915
8561916
8561871
8561869
8562329
8562328
8562327
8562318
8562317
8562316
8562315
8562314
8562313
8562307
8562306
8562305
8562304
8562303
8562302
8562301
8562300
8562299
8562298
8562297
8562296
8562295
8562341
8562340
8562339
8562338
8562337
8562336
8562324
8562326
8562325
8562312
8562311
8562310
8562309
8562308
8562335
8562334
8562333
8562332
8562331
8562330


8575065
8575124
8575066
8575127
8575128
8575067
8575036
8575037
8575069
8575038
8575039
8575070
8575060
8575040
8575041
8575042
8575043
8575044
8575045
8575046
8575047
8575048
8575049
8575050
8575051
8575052
8575053
8575054
8575055
8575056
8575057
8575087
8575117
8575118
8575072
8575073
8575074
8575075
8575076
8575077
8575078
8575079
8575080
8575081
8575082
8575083
8575084
8575086
8575085
8575091
8575088
8575089
8575090
8575094
8575092
8575093
8575106
8575095
8575096
8575097
8575098
8575099
8575100
8575101
8575102
8575103
8575104
8575105
8575107
8575108
8575109
8575110
8575111
8575112
8575059
8575068
8575071
8575061
8575062
8575167
8575168
8575169
8575170
8575171
8575173
8575161
8575162
8575174
8575163
8575175
8575207
8575176
8575164
8575178
8575177
8575179
8575180
8575165
8575181
8575182
8575183
8575166
8575184
8575185
8575186
8575187
8575188
8575189
8575202
8575203
8575204
8575205
8575206
8575208
8575209
8575210
8575211
8575212
8575172
8575213
8575214
8575223
8575215
8575216
8575217


8589245
8589244
8589268
8589267
8589263
8589266
8589265
8589264
8589235
8589262
8589261
8589260
8589272
8589243
8589240
8589242
8589241
8589239
8589238
8589258
8589278
8589277
8589290
8589276
8589303
8589288
8589318
8589317
8589289
8589316
8589315
8589314
8589313
8589312
8589311
8589310
8589309
8589308
8589307
8589306
8589305
8589304
8589287
8589302
8589301
8589300
8589299
8589298
8589297
8589296
8589294
8589293
8589295
8589292
8589291
8589286
8589285
8589284
8589283
8589282
8589281
8589280
8589279
8589548
8589547
8589545
8589546
8589544
8589543
8589542
8589541
8589540
8589539
8589538
8589537
8589536
8589535
8589534
8589533
8589532
8589531
8589563
8589562
8589561
8589560
8589559
8589558
8589557
8589556
8589555
8589554
8589552
8589551
8589550
8589549
8589569
8589568
8589567
8589566
8589565
8589564
8589522
8589521
8589520
8589519
8589518
8589517
8589516
8589515
8589514
8589513
8589512
8589511
8589510
8589509
8589508
8589507
8589526
8589525
8589524
8589523
8589553
8589570
8589506
8589505


8560499
8560500
8560502
8560503
8560504
8560505
8560501
8560506
8560507
8560508
8560509
8560510
8560511
8560512
8560513
8560514
8560515
8560516
8560518
8560519
8560520
8560521
8560522
8560523
8560525
8560526
8560527
8560528
8560529
8560530
8560531
8560532
8560533
8560517
8560534
8560524
8560535
8560536
8560537
8560538
8560539
8560540
8560490
8560491
8560492
8560493
8560495
8560496
8560494
8561784
8561785
8561786
8561787
8561788
8561789
8561790
8561791
8561792
8561793
8561794
8561795
8561782
8561783
8562144
8562130
8562143
8562142
8562141
8562140
8562139
8562138
8562137
8562136
8562135
8562134
8562133
8562132
8562131
8562170
8562169
8562168
8562153
8562167
8562166
8562146
8562165
8562164
8562163
8562162
8562161
8562160
8562159
8562158
8562129
8562157
8562156
8562155
8562154
8562152
8562151
8562150
8562149
8562148
8562147
8562202
8562172
8562171
8562201
8562200
8562199
8562198
8562197
8562196
8562195
8562194
8562193
8562192
8562204
8562191
8562190
8562189
8562188
8562187
8562186
8562185


8583107
8583106
8583105
8583104
8583103
8583102
8583101
8583100
8583099
8583098
8583097
8583096
8583095
8583094
8583093
8583092
8583133
8583132
8583131
8583130
8583129
8583128
8583127
8583126
8583125
8583124
8583123
8583122
8583121
8583120
8583119
8583118
8583117
8583114
8583116
8583115
8583113
8583112
8583111
8583154
8583153
8583152
8583151
8583150
8583149
8583134
8583148
8583147
8583146
8583145
8583144
8583143
8583142
8583141
8583140
8583139
8583138
8583137
8583136
8583135
8583164
8583175
8583174
8583173
8583172
8583171
8583170
8583169
8583168
8583167
8583166
8583165
8583163
8583161
8583160
8583162
8583159
8583158
8583157
8583156
8583155
8621601
8621600
8621599
8621598
8621597
8621596
8621595
8621594
8621593
8621592
8621649
8621648
8621647
8621646
8621645
8621644
8621641
8621640
8621639
8621638
8621637
8621636
8621635
8621634
8621633
8621632
8621631
8621630
8621629
8621627
8621626
8621625
8621624
8621623
8621622
8621620
8621621
8621619
8621617
8621618
8621616
8621615
8621614
8621613


8557480
8557479
8557478
8557477
8557476
8557475
8558001
8558009
8558008
8558006
8558005
8558004
8558003
8558002
8558019
8558007
8558018
8558017
8558016
8558015
8558014
8558013
8558012
8558011
8558010
8558027
8558026
8558025
8558024
8558039
8558038
8558037
8558023
8558022
8558033
8558036
8558035
8558034
8558020
8558021
8558032
8558031
8558030
8558029
8558028
8558202
8558201
8558208
8558209
8558207
8558206
8558205
8558204
8558203
8558217
8558216
8558215
8558214
8558213
8558212
8558211
8558210
8558847
8558848
8558849
8558850
8558851
8558852
8558853
8558854
8558855
8558856
8558857
8558858
8558859
8558860
8558861
8558862
8558863
8558831
8558864
8558826
8558827
8558828
8558829
8558830
8558832
8558833
8558834
8558835
8558836
8558837
8558838
8558839
8558840
8558841
8558846
8558842
8558843
8558844
8558845
8558887
8558886
8558888
8558889
8558890
8558891
8558892
8558893
8558894
8558895
8558896
8558897
8558898
8558899
8558900
8558901
8558865
8558866
8558867
8558868
8558869
8558870
8558871
8558872


8583228
8583229
8583230
8583231
8583232
8583233
8583234
8583235
8583236
8583237
8583238
8583239
8583240
8583241
8583242
8583243
8583244
8583245
8583246
8583247
8583248
8583249
8583250
8583251
8583252
8583253
8583254
8583255
8583256
8583257
8583258
8583259
8583218
8583219
8583220
8583221
8583725
8583724
8583731
8583753
8583752
8583751
8583750
8583749
8583748
8583747
8583746
8583713
8583712
8583759
8583758
8583757
8583715
8583714
8583730
8583729
8583728
8583727
8583726
8583756
8583755
8583754
8583723
8583722
8583721
8583720
8583719
8583718
8583717
8583745
8583744
8583735
8583741
8583740
8583739
8583738
8583743
8583742
8583716
8583736
8583734
8583733
8583732
8583737
8583766
8583765
8583791
8583794
8583793
8583792
8583790
8583789
8583788
8583787
8583786
8583785
8583784
8583796
8583783
8583782
8583781
8583780
8583764
8583763
8583762
8583761
8583760
8583806
8583799
8583800
8583804
8583803
8583802
8583801
8583805
8583798
8583797
8583795
8583779
8583778
8583777
8583776
8583775
8583774
8583773


8636250
8636254
8636241
8636255
8636584
8636585
8636586
8636587
8636573
8636589
8636575
8636590
8636594
8636606
8636591
8636592
8636593
8636597
8636598
8636923
8636932
8636933
8636934
8636936
8636937
8636938
8636926
8636927
8636928
8636953
8636954
8636955
8636941
8636942
8636945
8636946
8636947
8636950
8637315
8637316
8637310
8637311
8637304
8637305
8637313
8637306
8637307
8637308
8637302
8637291
8637303
8637293
8637294
8637296
8637297
8637301
8637325
8637326
8637327
8637328
8637324
8637329
8637342
8637332
8637331
8637333
8637336
8637335
8637322
8637321
8637339
8637319
8637340
8637317
8637341
8633114
8633102
8633115
8633104
8633105
8633106
8633116
8633107
8633109
8633110
8633111
8633112
8633113
8633096
8633098
8633099
8633097
8633100
8633101
8633121
8633122
8633124
8633125
8633135
8633127
8633128
8633129
8633130
8633131
8633117
8633132
8633118
8633133
8633134
8633119
8633120
8633688
8633690
8633691
8633684
8633685
8633686
8633687
8633689
8633694
8633698
8633693
8633695
8633696
8633697


8598174
8598177
8598166
8598175
8598167
8598168
8598176
8598169
8598170
8598165
8598171
8598172
8598173
8598178
8598180
8598179
8598181
8598315
8598320
8598322
8598321
8598304
8598305
8598306
8598307
8598316
8598317
8598308
8598309
8598310
8598311
8598312
8598313
8598314
8598319
8598318
8598331
8598332
8598335
8598336
8598337
8598338
8598339
8598340
8598330
8598341
8598333
8598334
8598342
8598323
8598324
8598325
8598327
8598328
8598329
8598326
8598516
8598531
8598517
8598532
8598544
8598533
8598518
8598534
8598519
8598535
8598520
8598536
8598528
8598537
8598521
8598538
8598522
8598539
8598529
8598540
8598547
8598542
8598548
8598543
8598541
8598545
8598523
8598546
8598530
8598524
8598525
8598526
8598527
8598570
8598571
8598562
8598572
8598579
8598575
8598564
8598576
8598550
8598580
8598563
8598549
8598565
8598582
8598551
8598566
8598567
8598578
8598568
8598583
8598552
8598553
8598569
8598554
8598573
8598555
8598556
8598560
8598557
8598574
8598558
8598561
8598559
8598581
8598577
8598711


In [26]:
len(codes)

21807

In [34]:
code_glimpse = trip2021.groupby(by="Trip Code", as_index=False).count().sort_values(by="Route", ascending=False)

code_glimpse=code_glimpse[["Trip Code", "Route"]]
code_glimpse.head()

,Trip Code,Route
20448,8621736,143
13453,8585846,143
20451,8621739,143
20440,8621728,143
20442,8621730,143


In [35]:
code_glimpse["Route"].unique()

array([143, 142, 141, 140, 138, 137, 136, 135, 134, 133, 132, 129, 128,
       127, 126, 125, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114,
       113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101,
       100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
        87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
        74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
        61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5], dtype=int64)

In [6]:
bus_stops = pd.read_csv(r'.\APC_Data\bge_busstops.csv')

In [8]:
bus_stops = bus_stops.drop(columns=["the_geom_2272", "the_geom"])

In [37]:
xsect = bus_stops.loc[bus_stops["category"] == "intersection"]
xsect

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,zone_id,wheelchair_boarding,dist,category
0,31613,Germantown Av & Erie Av,40.009403,-75.150954,NaN,NaN,1,1,49.827843,intersection
1,21261,Erie Av & Germantown Av,40.009233,-75.150671,NaN,NaN,1,1,54.570178,intersection
2,30285,Broad St & Erie Av - FS,40.009395,-75.151107,NaN,NaN,1,1,81.166948,intersection
3,32152,Erie Station - BSL,40.009234,-75.151155,NaN,NaN,50,2,87.992642,intersection
4,1188,Germantown Av & Erie Av,40.009019,-75.150660,NaN,NaN,1,1,112.802643,intersection
5,140,Erie Station - BSL,40.009234,-75.151250,NaN,NaN,50,2,114.005022,intersection
6,29964,Broad St & Erie Av,40.009368,-75.151261,NaN,NaN,1,1,118.394614,intersection
7,21260,Erie Av & Broad St,40.009208,-75.151439,NaN,NaN,1,1,167.787047,intersection
8,32424,Erie Av & Broad St - XH FS,40.009360,-75.151722,NaN,NaN,1,1,245.500924,intersection
9,17419,Erie Av & Elder St - FS,40.009053,-75.149844,NaN,NaN,1,1,294.962939,intersection


In [46]:
target = []

for name in xsect.stop_name:
    target.append(name)
    
print(target)

['Germantown Av & Erie Av', 'Erie Av & Germantown Av ', 'Broad St & Erie Av - FS', 'Erie Station - BSL', 'Germantown Av & Erie Av', 'Erie Station - BSL', 'Broad St & Erie Av', 'Erie Av & Broad St', 'Erie Av & Broad St - XH FS', 'Erie Av & Elder St - FS', 'Germantown Av & Broad St', 'Erie Av & Broad St - H FS']


In [22]:
 Erie = trip2021["Stop Seq."].str.contains("Erie", case=True)

In [24]:
German = trip2021["Stop Seq."].str.contains("German", case=True)

In [26]:
GermanErie= trip2021["Stop Seq."].str.contains("German|Erie", case=True)

In [28]:
GE = ['Germantown', 'Erie']

In [31]:
GElist = trip2021["Stop Seq."].apply(lambda sentence: all(word in sentence for word in GE))

In [38]:
XHErie = ['XH', 'Erie']

In [39]:
XHErielist = trip2021["Stop Seq."].apply(lambda sentence: all(word in sentence for word in XHErie))

In [51]:
targetlist = trip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in target))

In [57]:
BGE = trip2021[targetlist]
BGE.to_csv('BGE.csv', index=False)

In [55]:
BGE["Stop Seq."].unique()

array(['(043) - Broad St & Erie Av', '(039) - Broad St & Erie Av - FS',
       '(059) - Germantown Av & Broad St',
       '(060) - Germantown Av & Erie Av',
       '(004) - Germantown Av & Erie Av',
       '(042) - Germantown Av & Erie Av',
       '(015) - Erie Av & Germantown Av ',
       '(020) - Erie Av & Germantown Av ', '(038) - Erie Av & Broad St',
       '(039) - Erie Av & Elder St - FS', '(006) - Broad St & Erie Av',
       '(001) - Broad St & Erie Av', '(025) - Broad St & Erie Av - FS',
       '(026) - Broad St & Erie Av - FS',
       '(081) - Erie Av & Broad St - H FS',
       '(062) - Erie Av & Broad St - H FS',
       '(001) - Erie Av & Broad St - H FS',
       '(072) - Erie Av & Broad St - XH FS',
       '(066) - Erie Av & Broad St - XH FS',
       '(052) - Erie Av & Broad St - XH FS',
       '(001) - Erie Av & Broad St - XH FS'], dtype=object)